In [7]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import OneClassSVM
from sklearn.neighbors import LocalOutlierFactor, KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from scipy.sparse import csr_matrix
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV

# Ensure TensorFlow uses the GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    except RuntimeError as e:
        print(e)

# Load data with correct headers
column_names = [
    "duration", "protocol_type", "service", "flag", "src_bytes", "dst_bytes", "land",
    "wrong_fragment", "urgent", "hot", "num_failed_logins", "logged_in",
    "num_compromised", "root_shell", "su_attempted", "num_root", "num_file_creations",
    "num_shells", "num_access_files", "num_outbound_cmds", "is_host_login",
    "is_guest_login", "count", "srv_count", "serror_rate", "srv_serror_rate",
    "rerror_rate", "srv_rerror_rate", "same_srv_rate", "diff_srv_rate", "srv_diff_host_rate",
    "dst_host_count", "dst_host_srv_count", "dst_host_same_srv_rate",
    "dst_host_diff_srv_rate", "dst_host_same_src_port_rate", "dst_host_srv_diff_host_rate",
    "dst_host_serror_rate", "dst_host_srv_serror_rate", "dst_host_rerror_rate",
    "dst_host_srv_rerror_rate", "label", "difficulty_level"
]

train_df = pd.read_csv('Train.txt', delimiter=',', header=None, names=column_names)
test_df = pd.read_csv('Test.txt', delimiter=',', header=None, names=column_names)

# Function to convert dataset flags to Scapy-compatible flags
def convert_flag_to_scapy(flag):
    flag_translation = {
        'REJ': 'R',
        'SF': 'PA',
        'S0': 'S',
        'RSTO': 'R',
        'S1': 'S',
        'S2': 'S',
        'S3': 'S',
        'RSTOS0': 'R',
        'OTH': ''
    }
    return ''.join([flag_translation.get(f, '') for f in flag])

# Apply the flag conversion to the 'flag' column
train_df['flag'] = train_df['flag'].apply(convert_flag_to_scapy)
test_df['flag'] = test_df['flag'].apply(convert_flag_to_scapy)

# Convert the multi-class labels into binary labels: 1 for any attack, 0 for normal
train_df['binary_label'] = (train_df['label'] != 'normal').astype(int)
test_df['binary_label'] = (test_df['label'] != 'normal').astype(int)

# Selecting only the necessary features
features_to_use = ['protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes']
categorical_features = ['protocol_type', 'service', 'flag']
numerical_features = ['src_bytes', 'dst_bytes']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='drop'  # This drops the columns that are not explicitly transformed
)

# Apply preprocessing to both training and testing data
X_train = preprocessor.fit_transform(train_df[features_to_use])
X_test = preprocessor.transform(test_df[features_to_use])

# Convert to CSR format for models that require it
X_train_csr = csr_matrix(X_train)
X_test_csr = csr_matrix(X_test)

# Convert to dense format for deep learning models
X_train_dense = X_train_csr.toarray()
X_test_dense = X_test_csr.toarray()

# Save the preprocessor
with open('preprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)

# Labels for training and testing
y_train = train_df['binary_label']
y_test = test_df['binary_label']

# Dictionary to hold models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "One-Class SVM": OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1),
    "Local Outlier Factor": LocalOutlierFactor(n_neighbors=20, contamination=0.1),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

# Train and evaluate each model
for model_name, model in models.items():
    print(f"\nTraining {model_name}")
    start_time = time.time()  # Start time
    
    if model_name == "Local Outlier Factor":
        # For Local Outlier Factor, use fit_predict and then convert predictions
        y_pred = model.fit_predict(X_test_csr)
        y_pred = [1 if x == -1 else 0 for x in y_pred]  # Invert prediction for LOF
    elif model_name == "One-Class SVM":
        model.fit(X_train_csr)
        y_pred = model.predict(X_test_csr)
        y_pred = [1 if x == -1 else 0 for x in y_pred]  # Invert prediction for anomaly detection
    else:
        # For supervised learning models
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
    
    end_time = time.time()  # End time
    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")
    
    # Evaluate the classifier
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    
    # Measure inference time for 30 packets
    num_batches = X_test.shape[0] // 30
    inference_times = []
    for i in range(num_batches):
        start_inference = time.time()
        if model_name == "Local Outlier Factor":
            model.fit_predict(X_test_csr[i*30:(i+1)*30])
        else:
            model.predict(X_test[i*30:(i+1)*30])
        end_inference = time.time()
        inference_times.append(end_inference - start_inference)

    avg_inference_time = np.mean(inference_times)
    print(f"Average inference time per 30 packets: {avg_inference_time:.6f} seconds")

# Autoencoder for anomaly detection
def build_autoencoder(input_dim):
    input_layer = Input(shape=(input_dim,))
    encoder = Dense(16, activation="relu")(input_layer)
    encoder = Dense(8, activation="relu")(encoder)
    decoder = Dense(16, activation="relu")(encoder)
    decoder = Dense(input_dim, activation="sigmoid")(decoder)
    return Model(inputs=input_layer, outputs=decoder)

# Build and compile the autoencoder
input_dim = X_train_dense.shape[1]
autoencoder = build_autoencoder(input_dim)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# Train the autoencoder
start_time = time.time()  # Start time for autoencoder
autoencoder.fit(X_train_dense, X_train_dense, epochs=10, batch_size=32, validation_split=0.1, verbose=1)
end_time = time.time()  # End time for autoencoder
training_time = end_time - start_time
print(f"Autoencoder training time: {training_time:.2f} seconds")

# Evaluate the autoencoder
X_train_pred = autoencoder.predict(X_train_dense)
train_mse = np.mean(np.power(X_train_dense - X_train_pred, 2), axis=1)
threshold = np.percentile(train_mse, 95)

X_test_pred = autoencoder.predict(X_test_dense)
test_mse = np.mean(np.power(X_test_dense - X_test_pred, 2), axis=1)
y_pred = (test_mse > threshold).astype(int)

print("\nAutoencoder")
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))

# Measure inference time for 30 packets
num_batches = X_test_dense.shape[0] // 30
inference_times = []
for i in range(num_batches):
    start_inference = time.time()
    autoencoder.predict(X_test_dense[i*30:(i+1)*30])
    end_inference = time.time()
    inference_times.append(end_inference - start_inference)

avg_inference_time = np.mean(inference_times)
print(f"Average inference time per 30 packets: {avg_inference_time:.6f} seconds")

# Function to build the deep learning model for classification
def build_deep_model(neurons=32, activation='relu', optimizer='adam'):
    model = Sequential()
    model.add(Dense(neurons, input_dim=X_train_dense.shape[1], activation=activation))
    model.add(Dense(neurons // 2, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate deep learning model separately to debug
try:
    # Wrap the model with KerasClassifier
    model = KerasClassifier(model=build_deep_model, verbose=0, neurons=32, activation='relu', optimizer='adam', batch_size=32, epochs=10)
    
    # Fit the model
    model.fit(X_train_dense, y_train)
    
    # Predict and evaluate
    y_pred = (model.predict(X_test_dense) > 0.5).astype(int)
    
    print("\nDeep Learning Model")
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
    print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    
    # Measure inference time for 30 packets
    num_batches = X_test_dense.shape[0] // 30
    inference_times = []
    for i in range(num_batches):
        start_inference = time.time()
        model.predict(X_test_dense[i*30:(i+1)*30])
        end_inference = time.time()
        inference_times.append(end_inference - start_inference)
    
    avg_inference_time = np.mean(inference_times)
    print(f"Average inference time per 30 packets: {avg_inference_time:.6f} seconds")
except ValueError as e:
    print(f"Error during model training: {e}")

# Output for deep learning model




Training Random Forest
Training time: 4.36 seconds
Accuracy: 0.8225691980127751
ROC AUC Score: 0.8313252209616909
              precision    recall  f1-score   support

           0       0.74      0.89      0.81      9711
           1       0.91      0.77      0.83     12833

    accuracy                           0.82     22544
   macro avg       0.83      0.83      0.82     22544
weighted avg       0.84      0.82      0.82     22544

Average inference time per 30 packets: 0.003400 seconds

Training One-Class SVM
Training time: 125.98 seconds
Accuracy: 0.24174946770759403
ROC AUC Score: 0.24146601691879815
              precision    recall  f1-score   support

           0       0.19      0.24      0.21      9711
           1       0.30      0.24      0.27     12833

    accuracy                           0.24     22544
   macro avg       0.25      0.24      0.24     22544
weighted avg       0.25      0.24      0.24     22544

Average inference time per 30 packets: 0.006141 seconds
